In [3]:
import pandas as pd
import numpy as np
import os
from glob import glob
from tqdm import notebook

In [4]:
data_dir = './dataset'
merged_dir = './merged_dataset_log_volume'

In [5]:
def merge(idx):
    main_df_file = f'Totaldata_{idx}_ref_IXIC.csv'
    sub_df_file = f'supportindexscaled_{idx}_ref_IXIC.csv'
    
    main_df = pd.read_csv(os.path.join(data_dir, main_df_file), index_col="Date")
    sub_df = pd.read_csv(os.path.join(data_dir, sub_df_file), index_col="Date")
    
    merged_df = pd.merge(main_df, sub_df, on="Date")
    merged_df = merged_df.drop([merged_df.columns[0]], axis=1)
        
    merged_df = merged_df[120:]
    merged_df['Volume'] = np.log10(merged_df['Volume'])
    merged_df.rename(columns = {'Volume':'log_Volume'}, inplace=True)
#     test = not merged_df['Volume'].all()
#     if test:
#         print(merged_df[merged_df['Volume'] < 0])
#         input('')
    
    return merged_df

In [6]:
dataset = glob(data_dir + '/*.csv')
indices = []
for data_file in dataset:
    idx = data_file.split('_')[1]
    if idx not in indices:
        indices.append(idx)

In [7]:
prog_bar = notebook.tqdm(indices)
for idx in prog_bar:
    merged_df = merge(idx)
    merged_df.to_csv(os.path.join(merged_dir, f'merged_data_{idx}_ref_IXIC.csv'))


In [8]:
from simple_loader import get_data
prog_bar = notebook.tqdm(indices)

x_dataset, y_dataset, total_size = [], [], 0
for idx in prog_bar:
    merged_file = os.path.join(merged_dir, f'merged_data_{idx}_ref_IXIC.csv')
    data = get_data(n_days=32, data_path=merged_file, drop=['Date'])
    x_data, y_data = data
    
    total_size += x_data.nbytes + y_data.nbytes
    prog_bar.set_description(f'total_size = {total_size//(1024**2):^8d}MB')
    
    for i in range(x_data.shape[0]):
        x_dataset.append(x_data[i])
        y_dataset.append(y_data[i])

In [9]:
x_dataset = np.asarray(x_dataset)
y_dataset = np.asarray(y_dataset)
x_dataset.shape, y_dataset.shape

((6039128, 32, 20), (6039128, 1))

In [10]:
import pickle
import numpy as np

In [11]:
with open('finance_x_data_log.pickle', 'wb') as f:
    pickle.dump(x_dataset, f)

with open('finance_y_data_log.pickle', 'wb') as f:
    pickle.dump(y_dataset, f)

In [12]:
x_dataset, y_dataset = None, None

In [16]:
with open('finance_x_data_log.pickle', 'rb') as f:
    x_data = pickle.load(f)
    
with open('finance_y_data_log.pickle', 'rb') as f:
    y_data = pickle.load(f)

In [17]:
x_data.shape, y_data.shape

((6039128, 32, 20), (6039128, 1))

In [18]:
x_data, y_data = None, None